In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from tqdm import tqdm
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE

In [3]:
from os import walk

    
def best_blend_param_search(N_weight=5,N_search=10000):

    ho = pd.read_csv('../../data/Blend/ho/ho_label.csv')
    ho_y = ho['click'].values
    
    file_list = []
    for (_, _, f) in walk('../../data/Blend/ho/'):
        file_list.extend(f)
        break

    holdout_list = []
    score_list = []
    param_list = []
    ignore_list = ['wei_4262.csv','wei_4260.csv','ho_logistic.csv']
    file_list.remove('ho_label.csv')
    for each in ignore_list:
        file_list.remove(each)
    for each in file_list:
        df = pd.read_csv('../../data/Blend/ho/{}'.format(each))
        holdout_list.append(ho.merge(df,how='left',left_on='instance_id',right_on='instance_id')['predicted_score'].values)
    holdout_list = np.array(holdout_list)
    for i in tqdm(range(N_search)):
        param = np.random.randint(N_weight,size=(len(holdout_list)))
        preds = np.zeros(holdout_list.shape[1])
        w_sum = 0
        for index in range(len(param)):
            preds += param[index]*holdout_list[index]
            w_sum += param[index]
        if w_sum == 0:
            continue
        preds = preds/w_sum
        score_list.append(log_loss(ho_y,preds))
        param_list.append(param)
    score_seri = pd.Series(score_list)
    score_seri = score_seri.sort_values()    
    index = score_seri.index.values[0]
    best_score = score_seri.values[0]
    best_param = param_list[index]
    return_df = pd.DataFrame({'fileName':file_list,
                              'weights':best_param})
    return return_df,best_score

In [4]:
r,s = best_blend_param_search(N_weight=10,N_search=3000)
s

100%|██████████| 3000/3000 [00:19<00:00, 155.16it/s]


0.42497208017225974

In [87]:
r

,fileName,weights
0,nn_blend_0.42605738.csv,6
1,wei_4259.csv,5
2,shiyi_4258.csv,3
3,xgboost_baseline.csv,3


In [13]:
from os import walk

    
def best_blend_param_search_CV_noholdout(N_weight=5,N_search=10000):

    train_cv = pd.read_pickle(FILE.train_CV_label.value)
    y = train_cv.click.values
    
    file_list = []
    for (_, _, f) in walk('../../data/Blend/oof/'):
        file_list.extend(f)
        break

    cv_list = []
    score_list = []
    param_list = []
    ignore_list = ['wei_4262.csv','wei_4260.csv','ho_logistic.csv']
    for each in ignore_list:
        try:
            file_list.remove(each)
        except:
            pass
    for each in file_list:
        df = pd.read_csv('../../data/Blend/oof/{}'.format(each))
        cv_list.append(train_cv.merge(df,how='inner',on='instance_id')['predicted_score'].values)
    cv_list = np.array(cv_list)
    for i in tqdm(range(N_search)):
        param = np.random.randint(N_weight,size=(len(cv_list)))
        preds = np.zeros(cv_list.shape[1])
        w_sum = 0
        for index in range(len(param)):
            preds += param[index]*cv_list[index]
            w_sum += param[index]
        if w_sum == 0:
            continue
        preds = preds/w_sum
        score_list.append(log_loss(y,preds))
        param_list.append(param)
    score_seri = pd.Series(score_list)
    score_seri = score_seri.sort_values()    
    index = score_seri.index.values[0]
    best_score = score_seri.values[0]
    best_param = param_list[index]
    return_df = pd.DataFrame({'fileName':file_list,
                              'weights':best_param})
    return return_df,best_score

In [26]:
r,s = best_blend_param_search_CV_noholdout(N_weight=7,N_search=5000)
s

100%|██████████| 5000/5000 [09:57<00:00,  8.37it/s]


0.4147037519894541

In [28]:
r

,fileName,weights
0,ffm_train96W_oof.csv,1
1,nn_cv_blend_0.41721715.csv,4
2,wei4259.csv,2
3,xgb_train96W_oof.csv,6


In [24]:
r,s = best_blend_param_search_CV_noholdout(N_weight=7,N_search=500)
s

100%|██████████| 500/500 [01:01<00:00,  8.10it/s]


0.4146395254289823

In [25]:
r

,fileName,weights
0,nn_cv_tfidf_blend_0.41746588.csv,1
1,nn_cv_blend_0.41726551.csv,6
2,ffm_train96W_oof.csv,3
3,wei4259.csv,3
4,xgb_train96W_oof.csv,6
